# ANN 5x2 cv t-test

### Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Data Preprocessing

### Importing the dataset

In [ ]:
styleSilentShot = pd.read_excel('1.2 SilentSoundScale_asl_count30+.xlsx')
X = styleSilentShot.iloc[: , 1:-1].values # These are the predictor, independent variables (the 1:-1 excludes the first and last columns)
y = styleSilentShot.iloc[: , -1].values # These are the target variables (or classes, in this case)
styleSilentShot.head()

,Title,COUNTRY,ASL,RA,POV,INS,BCU,CU,MCU,MS,MLS,LS,VLS,Sound
0,10 Things I Hate About You,USA,6.7,58,4.0,2.0,64,224,82,37,36,53,3,1
1,"Adventures of Robin Hood, The",USA,5.0,0,0.0,0.0,3,53,71,77,109,111,76,1
2,"Affairs of Anatole, The",USA,8.0,23,16.0,9.0,33,26,89,141,136,70,5,0
3,Alley Cat,BRI,6.0,10,0.0,0.0,17,68,84,82,101,142,5,0
4,"Almost Perfect Affair, An",USA,4.2,64,12.0,7.0,70,199,93,51,26,51,9,1


In [ ]:
# print(X)

In [ ]:
# print(y)

### Encoding categorical data

One Hot Encoding the "COUNTRY" column

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[0.0 0.0 0.0 ... 36 53 3]
 [0.0 0.0 0.0 ... 109 111 76]
 [0.0 0.0 0.0 ... 136 70 5]
 ...
 [0.0 0.0 1.0 ... 99 66 40]
 [0.0 0.0 1.0 ... 123 243 26]
 [0.0 0.0 1.0 ... 83 396 21]]


### Splitting the dataset into the Training set and Test set

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
# random_state: Pass an int for reproducible output across multiple function calls

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train) # Feature scale even the one-hot-encoded variables for deep learning
X_test = sc.transform(X_test) # Not fitted to the test set to avoid information leakage
                              # Using the same scaler (fit mean and std dev) as the training data because 
                              # the test data is "unavailable"

In [ ]:
# print(X_test)

In [ ]:
# Feature scaling for hypothesis testing
X = sc.fit_transform(X)

# Construct Adam Scikit Learn Neural Network models

In [ ]:
from sklearn.neural_network import MLPClassifier

clf_adam_1 = MLPClassifier(solver='adam', alpha=0.01,
                     hidden_layer_sizes=(22), max_iter=31)
clf_adam_2 = MLPClassifier(solver='adam', alpha=0.01,
                     hidden_layer_sizes=(28), max_iter=31)

score1 = clf_adam_1.fit(X_train, y_train).score(X_test, y_test)
score2 = clf_adam_2.fit(X_train, y_train).score(X_test, y_test)

print('22-neuron Adam model prediction accuracy: %.2f%%' % (score1*100))
print('28-neuron Adam model prediction accuracy: %.2f%%' % (score2*100))

22-neuron Adam model prediction accuracy: 83.19%
28-neuron Adam model prediction accuracy: 73.45%


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (31) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (31) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# Implement the 5x2 cv paired t-test

H0: Both models have the same performance on the dataset.

H1: Both models doesn’t have the same performance on the dataset.

Paired-samples t-test:

1. If the result of the test suggests that there is insufficient evidence to reject the null hypothesis, then any observed difference in model skill is likely due to statistical chance.
2. If the result of the test suggests that there is sufficient evidence to reject the null hypothesis, then any observed difference in model skill is likely due to a difference in the models.

In [ ]:
from mlxtend.evaluate import paired_ttest_5x2cv

t, p = paired_ttest_5x2cv(estimator1=clf_adam_1,
                          estimator2=clf_adam_2,
                          X=X, y=y,
                          random_seed=1)

# If p > α (0.05), we cannot reject the null hypothesis and may conclude that the performance of the two algorithms is not significantly different
print('t statistic: %.3f' % t)
print('p value: %.3f' % p)

# interpret the result
if p <= 0.05:
	print('At a confidence of level of 0.95, the result of the test suggests that there is sufficient evidence to reject the null hypothesis. The difference between mean test auc performance is likely due to a difference in the models.')
else:
	print('At a confidence of level of 0.95, the result of the test suggests that there is insufficient evidence to reject the null hypothesis. The difference between mean test auc performance is likely due to statistical chance.')

# Construct L-BFGS Scikit Learn Neural Network models

In [ ]:
from sklearn.neural_network import MLPClassifier

clf_lbfgs_1 = MLPClassifier(solver='lbfgs', alpha=0.01,
                     hidden_layer_sizes=(22), max_iter=12)
clf_lbfgs_2 = MLPClassifier(solver='lbfgs', alpha=0.01,
                     hidden_layer_sizes=(28), max_iter=12)

score1 = clf_lbfgs_1.fit(X_train, y_train).score(X_test, y_test)
score2 = clf_lbfgs_2.fit(X_train, y_train).score(X_test, y_test)

print('22-neuron model prediction accuracy: %.2f%%' % (score1*100))
print('28-neuron model prediction accuracy: %.2f%%' % (score2*100))

22-neuron model prediction accuracy: 82.30%
28-neuron model prediction accuracy: 84.96%


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


# Implement the 5x2 cv paired t-test

In [ ]:
from mlxtend.evaluate import paired_ttest_5x2cv

t, p = paired_ttest_5x2cv(estimator1=clf_lbfgs_1,
                          estimator2=clf_lbfgs_2,
                          X=X, y=y,
                          random_seed=1)

# If p > α (0.05), we cannot reject the null hypothesis and may conclude that the performance of the two algorithms is not significantly different
print('t statistic: %.3f' % t)
print('p value: %.3f' % p)

# interpret the result
if p <= 0.05:
	print('At a confidence of level of 0.95, the result of the test suggests that there is sufficient evidence to reject the null hypothesis. The difference between mean test auc performance is likely due to a difference in the models.')
else:
	print('At a confidence of level of 0.95, the result of the test suggests that there is insufficient evidence to reject the null hypothesis. The difference between mean test auc performance is likely due to statistical chance.')